# Model

## Setup

In [15]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from tensorboardX import SummaryWriter

data = np.load('./dataset/preprocessed.npz')
train_x = torch.from_numpy(data['train_x']).float()
train_y = torch.from_numpy(data['train_y']).float()
test_x = torch.from_numpy(data['test_x']).float()

train_len = int(len(train_x) * 0.8)
eval_len = len(train_x) - train_len
train_data, eval_data = torch.utils.data.random_split(
    torch.utils.data.TensorDataset(train_x, train_y),
    [train_len, eval_len]
)
test_data = torch.utils.data.TensorDataset(test_x)

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('test_x shape:', test_x.shape)

train_x shape: torch.Size([3000, 20083])
train_y shape: torch.Size([3000])
test_x shape: torch.Size([4398, 20083])


## Define Model

In [16]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=20082, out_features=16384),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=16384, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=1024, out_features=256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=64, out_features=1)
        )
        self.loss_func = nn.MSELoss()
        
    def forward(self, in_data):
        return self.net(in_data[:,1:]) # skip id

    def loss(self, pred, truth):
        return self.loss_func(pred, truth)

## Runner function

In [17]:
def run(model, loaders, optimizer, writer, num_epoch=100, device='cpu'):
    def run_epoch(mode):
        epoch_loss = 0.0
        for i, batch in enumerate(loaders[mode], 0):
            in_data, labels = batch
            in_data, labels = in_data.to(device), labels.to(device)

            if mode == 'train':
                optimizer.zero_grad()

            logits = model(in_data)
            batch_loss = model.loss(logits, labels)

            epoch_loss += batch_loss.item()

            if mode == 'train':
                batch_loss.backward()
                optimizer.step()

        # sum of all batchs / num of batches
        epoch_loss /= i + 1
        
        print('epoch %d %s loss %.4f' % (epoch, mode, epoch_loss))
        # log to tensorboard
        if not (writer is None):
            writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={mode: epoch_loss}, 
                         global_step=epoch)
    for epoch in range(num_epoch):
        run_epoch('train')
        run_epoch('eval')

## Training

In [ ]:
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.99, 0.999))
run(
    model=model,
    loaders={
        'train': torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True),
        'eval': torch.utils.data.DataLoader(eval_data, batch_size=32, shuffle=True)
    },
    optimizer=optimizer, 
    writer=SummaryWriter('./logs/'), 
    num_epoch=10, 
    device='cpu'
)

epoch 0 train loss nan
epoch 0 eval loss nan
